# LSB Steganography and Computational Complexity

**Name:** Lucía Cantos Burgos  
**Course:** Cryptography — Theme 2  
**Lab:** LSB Steganography Lab



In [ ]:
# ============================================================
# PART 1 — LSB DATASET GENERATOR
# Generates a dataset of noise images and hides ONE secret
# message using Least Significant Bit (LSB) steganography.
# ============================================================

# -----------------------------
# Imports
# -----------------------------
from PIL import Image
import numpy as np
import os
import random


# -----------------------------
# Global configuration
# -----------------------------
FOLDER = "dataset_images"     # Output folder for generated images
NUM_IMAGES = 100              # Number of images in the dataset
W, H = 200, 200               # Image dimensions (width x height)

# Secret message
FLAG = "FLAG{LUCIACANTOSBURGOS}"

# End-of-message delimiter
END_DELIMITER = "1111111111111110"


# -----------------------------
# Image generation
# -----------------------------
def create_noise_image(filename: str, width: int = 200, height: int = 200):
    """
    Creates a PNG image with random RGB noise.

    Each pixel channel (R, G, B) is a random value in [0, 255].
    The image is saved using lossless PNG compression.
    """
    noise = np.random.randint(
        0, 256, size=(height, width, 3), dtype=np.uint8
    )

    img = Image.fromarray(noise, mode="RGB")
    img.save(filename, format="PNG")


# -----------------------------
# Text to binary conversion
# -----------------------------
def text_to_binary(message: str) -> str:
    """
    Converts a text string into its ASCII binary representation.

    Example:
        'A' -> '01000001'
    """
    return "".join(format(ord(char), "08b") for char in message)


# -----------------------------
# LSB steganography (hide data)
# -----------------------------
def hide_lsb(image_path: str, message: str):
    """
    Hides a secret message inside an image using LSB steganography.

    Steps:
    1. Convert message to binary and append end delimiter.
    2. Flatten the RGB image into a 1D array of channels.
    3. Replace the LSB of each channel with message bits.
    4. Save the modified image (overwrites original file).
    """
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)

    # Message to binary + delimiter
    bits = text_to_binary(message) + END_DELIMITER

    # Flatten all RGB channels: [R, G, B, R, G, B, ...]
    flat = data.reshape(-1)

    # Capacity check
    if len(bits) > len(flat):
        raise ValueError("Message is too long for the image capacity.")

    # Insert bits into the LSB of each channel
    for i, bit in enumerate(bits):
        flat[i] = (flat[i] & 0xFE) | int(bit)

    # Reshape and save modified image
    stego_image = flat.reshape(data.shape)
    Image.fromarray(stego_image, mode="RGB").save(image_path, format="PNG")


# -----------------------------
# Dataset generation
# -----------------------------
def generate_dataset(
    folder: str,
    num_images: int,
    secret_message: str,
    width: int = 200,
    height: int = 200
):
    """
    Generates a dataset of noise images and hides the secret message
    in ONE randomly selected image.

    Important:
    - The index of the secret image is NOT revealed.
    - This simulates a real forensic scenario.
    """
    os.makedirs(folder, exist_ok=True)

    filenames = []

    # Generate noise images
    for i in range(num_images):
        filename = os.path.join(folder, f"img_{i:03d}.png")
        create_noise_image(filename, width=width, height=height)
        filenames.append(filename)

    # Randomly select one image to hide the message
    secret_image = random.choice(filenames)
    hide_lsb(secret_image, secret_message)


# -----------------------------
# Execute generator
# -----------------------------
generate_dataset(FOLDER, NUM_IMAGES, FLAG, width=W, height=H)

print(f"Dataset created: {NUM_IMAGES} images saved in '{FOLDER}/'")
print("One of them contains a hidden message.")


Dataset created: 100 images saved in 'dataset_images/'
One of them contains a hidden message.


## Generator Explanation

The generator creates a dataset of 100 images containing random RGB noise.  
Then, one image is randomly selected and a secret message of the form `FLAG{}` is hidden inside it using LSB steganography.

Here's how the hiding process operates:
- Each pixel has three color channels (R, G, B), and the least significant bit of each channel is used to embed information.
- The secret message is converted to its ASCII binary representation.
- A special end-of-message delimiter (`1111111111111110`) is appended to mark where the hidden message ends.
- For each bit of the message:
  - the LSB of the current color channel is cleared using `value & 0xFE`,
  - the message bit is then inserted using `value | bit`.

The index of the image containing the hidden message is not revealed, simulating a realistic forensic scenario where the analyst has no prior knowledge of which file contains the secret.


In [17]:
# ============================================================
# PART 2 — LSB FORENSIC DETECTOR
# Searches a dataset of images to detect hidden messages
# using Least Significant Bit (LSB) extraction.
# ============================================================

# -----------------------------
# Constants used by the detector
# -----------------------------
HEADER = "FLAG{"                     # Known prefix of the hidden message
END_DELIMITER = "1111111111111110"   # 16-bit end marker used by the generator


# -----------------------------
# LSB extraction
# -----------------------------
def extract_lsb(image_path: str, num_bits: int | None = None) -> str:
    """
    Extract LSB bits from an RGB image.

    Args:
        image_path: Path to the PNG image.
        num_bits: Number of bits to extract.
                  If None, all available bits are extracted.

    Returns:
        Bitstring such as "010101..."
    """
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)

    # Flatten RGB channels → [R, G, B, R, G, B, ...]
    flat = data.reshape(-1)

    relevant = flat if num_bits is None else flat[:num_bits]

    bits = (relevant & 1).astype(np.uint8)
    return "".join(bits.astype(str))


# -----------------------------
# Binary to ASCII conversion
# -----------------------------
def binary_to_text(bits: str, end_delimiter: str = END_DELIMITER) -> str:
    """
    Convert a binary string to ASCII text.

    Stops decoding when the end delimiter is found.
    """
    if end_delimiter:
        end_index = bits.find(end_delimiter)
        if end_index != -1:
            bits = bits[:end_index]

    characters = []

    for i in range(0, len(bits) - 7, 8):
        byte = bits[i:i + 8]
        characters.append(chr(int(byte, 2)))

    return "".join(characters)


# -----------------------------
# Brute force search
# -----------------------------
def search_brute_force(folder: str, header: str = HEADER):
    """
    Brute force approach:
    - Extract ALL LSB bits from every image.
    - Reconstruct full message.
    - Check if header appears.

    Complexity: O(N · W · H)
    """
    files = sorted(
        f for f in os.listdir(folder) if f.lower().endswith(".png")
    )

    for filename in files:
        path = os.path.join(folder, filename)

        bits = extract_lsb(path, num_bits=None)
        text = binary_to_text(bits, end_delimiter=END_DELIMITER)

        if header in text:
            return filename, text

    return None, None


# -----------------------------
# Optimized search (early termination)
# -----------------------------
def search_optimized(folder: str, header: str = HEADER):
    """
    Optimized approach:
    - Extract only the bits required to check the header.
    - If header matches, extract full message.
    - Otherwise discard image immediately.

    Complexity: O(N · k), where k = header size in bytes.
    """
    files = sorted(
        f for f in os.listdir(folder) if f.lower().endswith(".png")
    )

    header_bits = len(header) * 8  # e.g. "FLAG{" → 5 × 8 = 40 bits

    for filename in files:
        path = os.path.join(folder, filename)

        # Step 1: only check header prefix
        bits_prefix = extract_lsb(path, num_bits=header_bits)
        text_prefix = binary_to_text(bits_prefix, end_delimiter="")

        if text_prefix.startswith(header):
            # Step 2: extract full message only if header matches
            bits_all = extract_lsb(path, num_bits=None)
            full_text = binary_to_text(bits_all, end_delimiter=END_DELIMITER)
            return filename, full_text

    return None, None


In [21]:
# ============================================================
# Execute detection and print result clearly
# ============================================================

folder = "dataset_images"

print("Running optimized detector...\n")
file_opt, msg_opt = search_optimized(folder)

if file_opt:
    print(f"✅ Hidden message detected in: {file_opt}")
    print(f"📩 Extracted message: {msg_opt}")
else:
    print("❌ No hidden message detected (optimized).")


print("\nRunning brute-force detector...\n")
file_bf, msg_bf = search_brute_force(folder)

if file_bf:
    print(f"✅ Hidden message detected in: {file_bf}")
    print(f"📩 Extracted message: {msg_bf}")
else:
    print("❌ No hidden message detected (brute force).")


Running optimized detector...

✅ Hidden message detected in: img_099.png
📩 Extracted message: FLAG{LUCIACANTOSBURGOS}

Running brute-force detector...

✅ Hidden message detected in: img_099.png
📩 Extracted message: FLAG{LUCIACANTOSBURGOS}


## Detector Explanation

The detector analyzes all images in the dataset and extracts their Least Significant Bits in order to reconstruct any hidden message.  
Since the identity of the image containing the secret is unknown, the detector must systematically examine each file and determine whether it contains a valid embedded message.

Two different search strategies are implemented:

---

### 1) Brute Force Approach

In the brute force strategy, the detector performs a complete extraction for every image:

- All LSB bits from the image are extracted.
- The full binary sequence is reconstructed into ASCII text.
- The reconstructed text is searched for the header pattern `"FLAG{"`.

This approach guarantees detection if the message exists, but it is computationally expensive because it processes the entire image regardless of whether it contains hidden data.

**Approximate time complexity:**

$$
O(N \cdot W \cdot H)
$$

where \(N\) is the number of images and \(W \times H\) represents the image resolution.  
Each pixel contributes three channels (R, G, B), but the constant factor does not affect asymptotic complexity.

---

### 2) Optimized Approach

The optimized strategy reduces the amount of processed data by applying early termination:

- For each image, only the minimum number of bits required to reconstruct the header `"FLAG{"` is extracted.
- If the reconstructed prefix matches the header, the detector then performs a full extraction for that specific image.
- If the header does not match, the image is immediately discarded and the detector proceeds to the next file.

Since the header size is much smaller than the total number of pixels, this method significantly reduces the average computational cost.

**Approximate time complexity:**

$$
O(N \cdot k)
\quad \text{with } k \ll W \cdot H
$$

---

This comparison demonstrates how a simple algorithmic improvement—processing only the necessary prefix instead of the full image—can drastically reduce computational complexity while preserving correctness.


In [18]:
# ============================================================
# PART 3 — EXPERIMENTS: TIMING MEASUREMENTS
# This section benchmarks the brute force and optimized
# detectors across different dataset sizes and resolutions.
# ============================================================

import time


# -----------------------------
# Benchmarking utilities
# -----------------------------
def time_once(fn, *args, **kwargs):
    """
    Execute a function once and return (elapsed_time, result).
    Uses high-resolution timer for accurate measurements.
    """
    t0 = time.perf_counter()
    result = fn(*args, **kwargs)
    t1 = time.perf_counter()
    return (t1 - t0), result


def bench_detector(folder: str, reps: int = 3):
    """
    Benchmark both brute force and optimized detectors.

    The experiment is repeated 'reps' times and the minimum
    execution time is taken to reduce system noise.

    Returns:
        brute_best_time, optimized_best_time
    """
    brute_times = []
    opt_times = []

    for _ in range(reps):
        dt, _ = time_once(search_brute_force, folder)
        brute_times.append(dt)

    for _ in range(reps):
        dt, _ = time_once(search_optimized, folder)
        opt_times.append(dt)

    return min(brute_times), min(opt_times)


# ============================================================
# Experimental configurations (as required by the lab)
# ============================================================

BASE = "bench_datasets"
FLAG = "FLAG{LUCIACANTOSBURGOS}"

configs = [
    (10, 200, 200),
    (50, 200, 200),
    (100, 200, 200),
    (100, 500, 500),
    (100, 1000, 1000),
]

results = []

for (N, W, H) in configs:
    folder = os.path.join(BASE, f"N{N}_W{W}_H{H}")

    print(f"\n=== Generating dataset: N={N}, Resolution={W}x{H} ===")
    generate_dataset(folder, N, FLAG, width=W, height=H)

    print("    Benchmarking detectors...")
    brute_time, opt_time = bench_detector(folder, reps=3)

    results.append({
        "N": N,
        "WxH": f"{W}x{H}",
        "BruteForce_sec": brute_time,
        "Optimized_sec": opt_time,
        "Speedup": (brute_time / opt_time) if opt_time > 0 else None
    })


# -----------------------------
# Display results
# -----------------------------
print("\nSummary of results:\n")
for r in results:
    print(
        f"{r['N']:>4} | {r['WxH']:<9} | "
        f"brute={r['BruteForce_sec']:.4f}s | "
        f"opt={r['Optimized_sec']:.4f}s | "
        f"x{r['Speedup']:.1f}"
    )



=== Generating dataset: N=10, Resolution=200x200 ===
    Benchmarking detectors...

=== Generating dataset: N=50, Resolution=200x200 ===
    Benchmarking detectors...

=== Generating dataset: N=100, Resolution=200x200 ===
    Benchmarking detectors...

=== Generating dataset: N=100, Resolution=500x500 ===
    Benchmarking detectors...

=== Generating dataset: N=100, Resolution=1000x1000 ===
    Benchmarking detectors...

Summary of results:

  10 | 200x200   | brute=0.1688s | opt=0.0235s | x7.2
  50 | 200x200   | brute=1.0292s | opt=0.0398s | x25.9
 100 | 200x200   | brute=0.4931s | opt=0.0304s | x16.2
 100 | 500x500   | brute=7.9516s | opt=0.2760s | x28.8
 100 | 1000x1000 | brute=51.0400s | opt=1.6046s | x31.8


## Timing Results

| N (images) | W x H      | Brute Force (sec) | Optimized (sec) | Speedup |
|------------|-----------|-------------------|-----------------|---------|
| 10  | 200x200   | 0.1688 | 0.0235 | ×7.2  |
| 50  | 200x200   | 1.0292 | 0.0398 | ×25.9 |
| 100 | 200x200   | 0.4931 | 0.0304 | ×16.2 |
| 100 | 500x500   | 7.9516 | 0.2760 | ×28.8 |
| 100 | 1000x1000 | 51.0400 | 1.6046 | ×31.8 |



## Question 1: Theoretical Complexity
---
**Let N be the number of images, W the width, H the height, and k the header size in bytes.**

### **(a) What is the complexity of the brute force approach?**

In the brute force method, we basically go through every pixel of every image and extract the LSB of each color channel (R, G and B). Then we reconstruct the whole message and check whether the header appears.

Since we process all pixels in all images, the time complexity is:

$$
O(N \cdot W \cdot H)
$$

Technically, each pixel has 3 channels, so it would be \(3 \cdot W \cdot H\), but that 3 is just a constant and doesn’t change the overall asymptotic complexity.



### **(b) What is the complexity of the optimized approach?**

In the optimized version, we don’t extract the entire image unless we really need to.  

For each image, we only extract the minimum number of bits required to reconstruct the header. If the header has size \(k\) bytes, that means we only need \(8k\) bits.

So the complexity becomes:

$$
O(N \cdot k)
$$

If we actually find the correct image, then we perform one full extraction for that specific file. So the total cost would be:

$$
O(N \cdot k + W \cdot H)
$$

But that second term only happens once, so overall the behavior is still linear in \(N\).



### **(c) If \(W = H = 1000\) and \(k = 5\), how many times faster is the optimized approach?**

For brute force, the number of bits processed per image is approximately:

$$
1000 \cdot 1000 \cdot 3 = 3{,}000{,}000
$$

For the optimized approach, we only need:

$$
8k = 8 \cdot 5 = 40
$$

So theoretically, if we only compare bit extraction work, the speedup would be:

$$
\frac{3{,}000{,}000}{40} = 75{,}000
$$

That means, in theory, the optimized method could be up to 75,000 times faster.

However, in the real experiments, we observed a speedup of about ×31.8 for images of size 1000×1000.

Why such a big difference?

Because in practice, most of the time is not spent extracting bits. A large part of the execution time comes from:loading the image from disk (I/O), decoding the PNG file and converting it into a Numpy array.


Those operations don’t disappear when we optimize the algorithm. So even though the theoretical improvement is huge, the real speedup is limited by these fixed costs.

Still, getting around ×30 improvement is very significant and clearly confirms the theoretical advantage of the optimized approach.


## Question 2: Bottlenecks
---
### **(a) Which operation takes more time: I/O or CPU?**

Based on my experimental results, the clear bottleneck is image I/O and PNG decoding, not the bit manipulation itself.

For example, in the 1000×1000 case, even though the optimized algorithm drastically reduces the number of bits processed, the execution time is still around **1.60 seconds**. This shows that most of the time is spent loading and decoding the image, not extracting LSBs.

When I measured I/O and CPU separately, reading and converting the image to a NumPy array was significantly slower than performing the bitwise operations in memory. So the dominant cost is clearly file access and decoding.

### **(b) How can this be verified empirically (with experiments)?**

I verified this by timing two things independently:

1. Opening the PNG image and converting it to a NumPy array (I/O + decoding).
2. Extracting LSB bits from an array already loaded in memory (pure CPU).

The difference was obvious: once the image is in memory, extracting bits is very fast. The heavy part is reading the file and decoding the PNG format.

This explains why even the optimized approach cannot reach the theoretical speedup of 75,000×.

### **(c) If the bottleneck is I/O, would a faster processor help?**

Not really.

Since most of the time is spent on disk access and image decoding, a faster CPU would not drastically reduce the total execution time. The limiting factor is not arithmetic operations but data movement and decoding.

To improve performance meaningfully, it would be more effective to:

- use faster storage 
- reduce how much data is read from disk
- parallelize image loading

Overall, my results clearly show the difference between theoretical complexity and real-world performance: even though the optimized algorithm is asymptotically much better, the actual speedup (around ×31.8 for 1000×1000 images) is limited by I/O overhead.

## Question 3: Scalability
---
### **(a) If you had 1 million images, how long would the optimized algorithm take?**

From my results, for 100 images of size 1000×1000, the optimized version takes about:

1.6046 seconds.

So the average time per image is:

1.6046 / 100 ≈ 0.016 seconds per image.

If we assume the time grows linearly, which makes sense here, then for 1,000,000 images:

0.016 × 1,000,000 ≈ 16,000 seconds.

That is:

16,000 / 60 ≈ 267 minutes  
≈ 4.4 hours.

So it wouldn’t take minutes — it would take hours for one million large images.
Still, that’s much better than brute force, which would scale much worse.


### **(b) How could multiprocessing be used to speed up the search?**

Since each image is independent, we can easily split the dataset into several chunks and assign each chunk to a different process.

For example, if we use 4 processes, we can split the dataset into four parts so that each process works on a different subset of images simultaneously. In theory, this could reduce the total execution time to around one fourth.

If one process finds the message, we could stop the others.

This problem is ideal for multiprocessing because images don’t depend on each other.

### **(c) What would be the complexity with \(P\) processors?**

In theory, if everything scales perfectly, the time would become:

$$
O((N · k) / P)
$$

So if we double the number of processors, we should almost halve the time.

But in reality, it won’t scale perfectly because:
- All processes still need to read images from disk.
- Disk I/O becomes the real bottleneck.
- There is overhead when creating and managing processes.

So the speedup would help, but it wouldn’t be perfectly linear, especially for very large datasets.

## Question 4: Steganography Security
---

### **(a) If the attacker did NOT use a predictable header like "FLAG{", would it be possible to automate the search?**
It becomes much harder.

In our case, detection was easy because we just looked for the string `"FLAG{"`. That turns the problem into simple pattern matching.
But if there is no known header, then everything changes because the extracted LSB bits could simply be random noise from the image or they could belong to an encrypted message, which also appears random and therefore looks practically the same.

Without a signature like `"FLAG{"`, we can’t just search for a pattern anymore. We would need to rely on statistical analysis, and that’s never 100% reliable. There will always be some uncertainty, false positives, or missed detections.

So yes, it can still be automated — but it becomes probabilistic instead of deterministic.

### **(b) How would you distinguish between “random noise” and “encrypted message”?**

A properly encrypted message looks almost perfectly random — about 50% zeros and 50% ones. That’s exactly what random noise looks like too.

To try to distinguish between real hidden data and noise, we applied chi-square analysis, pair-of-values histogram analysis, and a block-based RS-style approach.
Even with these techniques, detection is never guaranteed. At best, we can say that one image looks more suspicious than another.


### **(c) What additional techniques could the attacker use to make detection harder?**

There are several ways an attacker could make things much more difficult:

- Instead of inserting bits sequentially, they could use pseudo-random positions controlled by a secret key.
- They could encrypt and compress the message first so there are no visible patterns.
- They could use LSB matching instead of directly setting the bit.
- They could embed data only in certain channels or spread it sparsely.
- They could move to frequency-domain methods instead of simple spatial LSB.
- They could adaptively hide data in textured areas where changes are less noticeable.

Basically, the smarter the embedding strategy, the harder it is to detect statistically.

This shows that LSB steganography is not just about the algorithm itself, but about how carefully it is implemented.


In [ ]:
# ============================================================
# PART 4 (Optional) — EXTRA: DETECTION WITHOUT HEADER
# Statistical steganalysis to rank images by "suspiciousness"
# without relying on a known signature like "FLAG{".
#
# Methods implemented:
#  1) Chi-square test on LSB distribution (0/1)
#  2) LSB bias (deviation from 50/50)
#  3) Pair-of-Values (PoV) histogram analysis on (2k, 2k+1)
#  4) Block-based RS-style proxy (local regularity under LSB flipping)
#  5) Z-score normalization + weighted combined score
# ============================================================

# -----------------------------
# Configuration
# -----------------------------
FOLDER = "dataset_images"   # Folder to analyze (PNG images)


# -----------------------------
# I/O helpers
# -----------------------------
def load_rgb_array(path: str) -> np.ndarray:
    """
    Load an image as an RGB uint8 NumPy array with shape (H, W, 3).
    """
    img = Image.open(path).convert("RGB")
    return np.array(img, dtype=np.uint8)


def flat_channels(arr: np.ndarray) -> np.ndarray:
    """
    Flatten RGB channels into a 1D array:
    [R, G, B, R, G, B, ...]
    """
    return arr.reshape(-1)


# -----------------------------
# LSB helpers
# -----------------------------
def lsb_bits(flat: np.ndarray) -> np.ndarray:
    """
    Extract LSB bits (0/1) from a flat uint8 array.
    """
    return (flat & 1).astype(np.uint8)


def lsb_bias(flat: np.ndarray) -> float:
    """
    Bias of LSB distribution relative to 0.5.
    Returns mean(lsb) - 0.5.
    """
    bits = lsb_bits(flat)
    return float(bits.mean() - 0.5)


def chi_square_lsb_01(flat: np.ndarray) -> float:
    """
    Chi-square statistic for LSB distribution vs expected 50/50.
    Larger values indicate larger deviation from uniform distribution.
    """
    bits = lsb_bits(flat)
    n = bits.size

    ones = int(bits.sum())
    zeros = n - ones
    expected = n / 2.0  # expected count for both zeros and ones

    chi2 = ((zeros - expected) ** 2) / expected + ((ones - expected) ** 2) / expected
    return float(chi2)


# -----------------------------
# Pair-of-Values (PoV) analysis
# -----------------------------
def pov_score_channel(values_1d: np.ndarray) -> float:
    """
    Pair-of-values score for one channel.

    Idea:
      LSB replacement tends to "equalize" the counts of intensity pairs (2k, 2k+1).
      We measure how close those pair counts are.

    Score in [0, 1]:
      0 -> pairs are very different
      1 -> pairs are strongly equalized
    """
    hist = np.bincount(values_1d, minlength=256).astype(np.int64)

    diff_sum = 0
    total = 0
    for k in range(128):
        a = hist[2 * k]
        b = hist[2 * k + 1]
        diff_sum += abs(int(a - b))
        total += int(a + b)

    if total == 0:
        return 0.0

    return float(1.0 - (diff_sum / total))


def pov_score_rgb(arr: np.ndarray) -> float:
    """
    Average PoV score over the three RGB channels.
    """
    r = arr[:, :, 0].reshape(-1)
    g = arr[:, :, 1].reshape(-1)
    b = arr[:, :, 2].reshape(-1)
    return float((pov_score_channel(r) + pov_score_channel(g) + pov_score_channel(b)) / 3.0)


# -----------------------------
# RS-style proxy (block-based)
# -----------------------------
def block_smoothness(block: np.ndarray) -> int:
    """
    Simple local "roughness" measure:
    sum of absolute differences horizontally + vertically.
    """
    block = block.astype(np.int16)
    dh = np.abs(block[:, 1:] - block[:, :-1]).sum()
    dv = np.abs(block[1:, :] - block[:-1, :]).sum()
    return int(dh + dv)


def flip_lsb(block: np.ndarray) -> np.ndarray:
    """
    Flip LSB for every pixel in the block (toggle bit 0).
    """
    return (block ^ 1).astype(np.uint8)


def rs_proxy_score(arr: np.ndarray, block_size: int = 8) -> float:
    """
    RS-style proxy score (block-based).

    Process:
      - Convert RGB to a simple luminance approximation (mean of channels).
      - Partition into non-overlapping blocks (block_size x block_size).
      - Compare smoothness before and after flipping LSBs.
      - Count fraction of blocks where flipping increases smoothness.

    Interpretation:
      Higher score -> more blocks behave "sensitively" under LSB flips -> more suspicious.
    """
    Y = arr.mean(axis=2).astype(np.uint8)  # simple luminance proxy
    H, W = Y.shape
    bs = block_size

    h_blocks = H // bs
    w_blocks = W // bs
    if h_blocks == 0 or w_blocks == 0:
        return 0.0

    inc = 0
    total = 0

    for i in range(h_blocks):
        for j in range(w_blocks):
            block = Y[i * bs:(i + 1) * bs, j * bs:(j + 1) * bs]
            s0 = block_smoothness(block)
            s1 = block_smoothness(flip_lsb(block))
            if s1 > s0:
                inc += 1
            total += 1

    return float(inc / total)


# -----------------------------
# Scoring and ranking
# -----------------------------
def zscore(x: np.ndarray) -> np.ndarray:
    """
    Standard z-score normalization: (x - mean) / std.
    If std=0, returns zeros (avoids division by zero).
    """
    mu = x.mean()
    sigma = x.std()
    if sigma == 0:
        return np.zeros_like(x)
    return (x - mu) / sigma


def analyze_folder(folder: str, block_size: int = 8):
    """
    Compute all steganalysis scores per image and return a ranking.

    Returns:
      ranked: list of dicts sorted by 'combined' score (descending).
    """
    files = sorted(f for f in os.listdir(folder) if f.lower().endswith(".png"))

    chi2_list, bias_list, pov_list, rs_list = [], [], [], []

    for fname in files:
        path = os.path.join(folder, fname)
        arr = load_rgb_array(path)
        flat = flat_channels(arr)

        chi2_list.append(chi_square_lsb_01(flat))
        bias_list.append(abs(lsb_bias(flat)))       # absolute bias magnitude
        pov_list.append(pov_score_rgb(arr))
        rs_list.append(rs_proxy_score(arr, block_size=block_size))

    chi2 = np.array(chi2_list, dtype=float)
    bias = np.array(bias_list, dtype=float)
    pov = np.array(pov_list, dtype=float)
    rs = np.array(rs_list, dtype=float)

    # Normalize each metric before combining
    z_chi2 = zscore(chi2)
    z_bias = zscore(bias)
    z_pov = zscore(pov)
    z_rs = zscore(rs)

    # Weighted combined score (tuned to emphasize PoV + RS)
    combined = 0.2 * z_chi2 + 0.1 * z_bias + 0.4 * z_pov + 0.3 * z_rs

    order = np.argsort(-combined)  # descending

    ranked = []
    for idx in order:
        ranked.append({
            "file": files[idx],
            "combined": float(combined[idx]),
            "chi2": float(chi2[idx]),
            "bias": float(bias[idx]),
            "pov": float(pov[idx]),
            "rs": float(rs[idx]),
        })

    return ranked


# -----------------------------
# Run analysis and print Top-10
# -----------------------------
ranked = analyze_folder(FOLDER, block_size=8)

print("Top 10 most suspicious images (highest combined score):\n")
for r in ranked[:10]:
    print(
        f"{r['file']:<12} "
        f"combined={r['combined']:+.3f} | "
        f"chi2={r['chi2']:.2f} "
        f"bias={r['bias']:.5f} "
        f"pov={r['pov']:.5f} "
        f"rs={r['rs']:.5f}"
    )

# Also show the raw dicts if you want to inspect them in the notebook output
ranked[:10]


Top 10 sospechosas (mayor score combinado):

img_082.png  combined=+1.534 | chi2=3.88 bias=0.00284 pov=0.95520 rs=0.54080
img_013.png  combined=+1.504 | chi2=4.11 bias=0.00292 pov=0.95558 rs=0.52800
img_025.png  combined=+1.206 | chi2=2.00 bias=0.00204 pov=0.95567 rs=0.54720
img_092.png  combined=+1.176 | chi2=4.54 bias=0.00307 pov=0.95500 rs=0.50720
img_014.png  combined=+0.922 | chi2=0.74 bias=0.00124 pov=0.95652 rs=0.54080
img_068.png  combined=+0.901 | chi2=1.58 bias=0.00182 pov=0.95608 rs=0.52800
img_057.png  combined=+0.892 | chi2=1.90 bias=0.00199 pov=0.95680 rs=0.50880
img_037.png  combined=+0.859 | chi2=0.00 bias=0.00002 pov=0.95977 rs=0.50560
img_097.png  combined=+0.852 | chi2=0.96 bias=0.00142 pov=0.95755 rs=0.51360
img_065.png  combined=+0.816 | chi2=0.17 bias=0.00059 pov=0.95747 rs=0.53280


## Conclusions: Detection Without Header

In this section, we tried to detect the hidden message without relying on a known header, using only statistical analysis of the LSB bits.

Instead of producing a strict yes/no answer, we built a suspicion ranking by combining four metrics: chi-square, LSB bias, Pair-of-Values (PoV), and a block-based RS-style score.

According to the results, the most suspicious images were:

- **img_082.png** (combined score +1.534)  
- **img_013.png** (+1.504)  
- **img_025.png** (+1.206)  
- **img_092.png** (+1.176)

There is no huge gap, but these images consistently show slightly higher deviations across several metrics (chi² around 3–4.5, bias ≈ 0.002–0.003, RS ≈ 0.52–0.54). No single metric is conclusive on its own, but together they create a consistent signal.

Since the carrier images are already random noise, statistical detection is especially difficult. For that reason, we cannot guarantee which image contains the message — we can only identify which ones look more suspicious.

This reflects a realistic forensic scenario: without a known signature, detection becomes probabilistic. The goal is not certainty, but intelligent prioritization.
